In [ ]:
import pandas as pd

# Load the files
file_1 = pd.read_csv('file_1.csv')
file_2 = pd.read_csv('file_2.csv')

# Ensure 'Tags' column exists in file_2
file_2['Tags'] = None

# Split items in 'Headers_Contains' and 'Subject_Not_RE_FW' by commas
file_1['Headers_Contains'] = file_1['Headers_Contains'].str.split(',')
file_1['Subject_Not_RE_FW'] = file_1['Subject_Not_RE_FW'].str.split(',')

# Define a function to check conditions and assign tags
def assign_tags(row):
    tags = []
    for _, rule in file_1.iterrows():
        # Check Headers_Contains
        if any(header.strip() in row['Headers'] for header in rule['Headers_Contains']):
            # Condition 1: Headers_Contains match
            tags.append(rule['Tag'])

            # Condition 2: Subject_Not_RE_FW not in Subject
            if all(subj.strip() not in row['Subject'] for subj in rule['Subject_Not_RE_FW']):
                tags.append(rule['Tag'])

                # Condition 3: Subject_Contains in Subject
                if rule['Subject_Contains'] in row['Subject']:
                    tags.append(rule['Tag'])

                # Condition 4: Subject_Not_Contains not in Subject
                if rule['Subject_Not_Contains'] not in row['Subject']:
                    tags.append(rule['Tag'])

    return ', '.join(set(tags)) if tags else None

# Apply function to each row in file_2
file_2['Tags'] = file_2.apply(assign_tags, axis=1)

# Save the updated file_2
file_2.to_csv('file_2_with_tags.csv', index=False)
